In [4]:
#hand
##Mediapipe mp4 Motion detection
import cv2
import mediapipe as mp
import numpy as np
import json

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
# 從此處決定要偵測甚麼
# 姿势估计：
# mediapipe.solutions.pose：姿势估计模块，用于检测人体关键点的姿势。
# 手部追踪：
# mediapipe.solutions.hands：手部追踪模块，用于检测手部的关键点。
# 面部检测：
# mediapipe.solutions.face_detection：面部检测模块，用于检测和定位人脸。
# Holistic（整体身体）模型：
# mediapipe.solutions.holistic

mp_hands = mp.solutions.hands
# files
TrackName = ["testing","Arlequin", "Eusebins", "Floresla", "Pierrot", "Valse Nobles"]
VideoType = ["","_above", "_side","_short"]
file_name=TrackName[3]+VideoType[1]


Date="1020"
video_folder="./sample/videos/"+Date+"/"
# open MP4 file
video_file =video_folder+file_name +".mp4"
print(video_file)
cap = cv2.VideoCapture(video_file)
frame_rate = cap.get(cv2.CAP_PROP_FPS)
print("frame rate: ",frame_rate)

# 待寫入的JSON 文件
json_file = "./sample/json/"+file_name+".json"
json_data = []
     
with mp_hands.Hands(
    min_detection_confidence=0.3,
    max_num_hands=2,
    min_tracking_confidence=0.3) as hands:
    if not cap.isOpened():
        print("Cannot open video file")
        exit()
    frameID=0
    loop=1
    while loop:    
        ret, img = cap.read()
        if not ret:
            print("Cannot receive frame")
            break
        img = cv2.resize(img, (1280, 720))
        img2 = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = hands.process(img2)
        try:
            condition = np.stack((results.segmentation_mask,) * 3, axis=-1) > 0.1
        except:  # noqa: E722
            pass
        
        #print("frame:",frameID)
        #frame_data = {"Frame#": frameID, "landmarks": []}
        frame_data = {"Frame#": frameID, "Time (s)": frameID / frame_rate, "landmarks": []}

        
        
        handID = 0  # 0,1
        hand_dict = {0: 'left hand', 1: 'right hand',2:'unknow'}
        # results.multi_hand_landmarks共兩隻手 
        # results.multi_hand_world_landmarks: (世界座標) 
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
    #             print(hand_dict[handID])
                landmarks_data = {"hand": hand_dict[handID], "landmarks": []}
                #一隻手 hand_landmarks.landmark 手部共21個節點
                landmarksID = 0  # 0~20  
                for landmark in hand_landmarks.landmark:
    #                 print("landmarks #",landmarksID)
    #                 print(f'{landmark.x}, {landmark.y}, {landmark.z}')
                    landmarks_data["landmarks"].append({
                        "landmarks #": landmarksID,
                        "landmark.x": landmark.x,
                        "landmark.y": landmark.y,
                        "landmark.z": landmark.z
                    })
                    landmarksID += 1
                frame_data["landmarks"].append(landmarks_data)
                mp_drawing.draw_landmarks(
                    img,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style())
                handID += 1
            json_data.append(frame_data)
        else:
            pass
        cv2.imshow('Mediapipe_Motion', img)
#         loop=0
        frameID+=1
        if cv2.waitKey(5) == ord('q'):
            break
with open(json_file, "w") as file:
    json.dump(json_data, file, indent=4)
cap.release()
cv2.destroyAllWindows()

./sample/videos/1020/Floresla_above.mp4
frame rate:  60.0


# Labeling

In [1]:
import pandas as pd
import json

# 讀取情緒標記表格數據
el_file = 'emotion_labels.csv'  # 替換為你的表格文件名
el_data = pd.read_csv(el_file, delimiter='\t')  # 使用適當的分隔符
# # files
# TrackName = ["Arlequin", "Eusebins", "Floresla", "Pierrot", "Valse Nobles"]
# VideoType = ["above", "side","short"]
# file_name=TrackName[0]+"-"+VideoType[0]
# # 待寫入的JSON 文件
# json_file = "./json/"+file_name+".json"
# json_data = []

# 載入已生成的JSON文件
input_json_file = json_file 
print("Start labeling",input_json_file)

# 讀取JSON文件
with open(input_json_file, 'r') as file:
    data = json.load(file)


# 創建一個新的JSON數據結構，用於存儲更新後的情緒標記
new_data = []
#用來確定是否在第一個時間段
in_first_time_stamp=1
# csv中的時間點
case_list=[]
# 遍歷json每一帧的數據，並添加情緒標記
for frame_data in data:
    frame_time = frame_data['Time (s)']  # 獲取.json 帧的時間
    for i in range(len(el_data['Time Stamp'])):#找時間段
        if i>(len(el_data['Time Stamp'])-1):
            case3=">"+str(el_data['Time Stamp'][i])
            print(case3)
            case_list.append(case3)
            current_timestamp=el_data  
            frame_data['Valence'] = int(current_timestamp['Valence'][i])
            frame_data['Arousal'] = int(current_timestamp['Arousal'][i])    
            new_data.append(frame_data)
            break
        else:
            if(el_data['Time Stamp'][i]< frame_time <el_data['Time Stamp'][i+1]):
                case1= str(el_data['Time Stamp'][i])+"~"+str(el_data['Time Stamp'][i+1])
                print(case1)
                case_list.append(case1)
                current_timestamp=el_data   
                frame_data['Valence'] = int(current_timestamp['Valence'][i] )
                frame_data['Arousal'] = int(current_timestamp['Arousal'][i] )
                new_data.append(frame_data) 
                break
            elif(frame_time <el_data[ 'Time Stamp'][i]):
                case2= "0 ~"+str(el_data['Time Stamp'][i])
                print(case2)
                case_list.append(case2)
                if i>1:
                    current_timestamp=el_data
                    frame_data['Valence'] = int(current_timestamp['Valence'][i-1])
                    frame_data['Arousal'] = int(current_timestamp['Arousal'][i-1]) 
                else:#head
                    current_timestamp=0
                    frame_data['Valence'] = 0
                    frame_data['Arousal'] = 0
                new_data.append(frame_data) 
                break
            
             
            
#創建新的JSON檔案
case_file= "./json/case.json"
with open(case_file, 'w') as file:
    json.dump(case_list, file, indent=4)
output_json_file = "./json/"+file_name+'_labeled.json'
with open(output_json_file, 'w') as file:
    json.dump(new_data, file, indent=4)

print("emotion labek JSON file Generated：", output_json_file)

FileNotFoundError: [Errno 2] No such file or directory: 'emotion_labels.csv'

In [3]:
# el_data.iloc[0]["Time Stamp"]

8